## E-Grocery Recommendation

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules , fpgrowth , fpmax
from sklearn.metrics import pairwise_distances
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
import scipy.sparse as sparse
import implicit

In [3]:
data=pd.read_csv("dataset/Customer_Data.csv")

In [17]:
data.head()

,order_id,customer_name,customer_emailid,order_date,order_fruits,order_vegetables,order_milk,order_rice,basket
0,a1,CUST189,CUST189@gmail.com,1/1/2018,"Black sapota-1kg, Custard apple-1kg, Kamiya pa...","Gaajar-200g, Brocoli-250g",NaN,NaN,"Black sapota-1kg, Custard apple-1kg, Kamiya pa..."
1,a2,CUST230,CUST230@gmail.com,1/1/2018,"Soursop Custard apple-1kg, Guinea Gold papaya-...","brown onion-5kg, Tomato Hybrid-2kg, red chilli...",NaN,NaN,"Soursop Custard apple-1kg, Guinea Gold papaya-..."
2,a3,CUST289,CUST289@gmail.com,1/1/2018,"Green sapota-1kg, Soursop Custard apple-1kg, K...","brown onion-5kg, cherrie tomato-1kg, red bell ...",Buffalo Milk-1.5L,"Basmati Rice-3kg, Brown Rice-5kg","Green sapota-1kg, Soursop Custard apple-1kg, K..."
3,NaN,CUST193,CUST193@gmail.com,2/1/2018,"Dasheri mango-0.5kg, Dholka pomograntes-1kg, ...","Tomato Local-3kg, cherrie tomato-1kg, red chil...",Buffalo Milk-1.5L,Brown Rice-5kg,"Dasheri mango-0.5kg, Dholka pomograntes-1kg, ..."
4,a5,CUST234,CUST234@gmail.com,2/1/2018,NaN,"Spring onion-250g, Tomato Hybrid-2kg, cherrie ...",Buffalo Milk-1.5L,Basmati Rice-3kg,"Spring onion-250g, Tomato Hybrid-2kg, cherrie ..."


**9 columns in the dataset**
* order_id - Everyone transaction has a unique ID generated 
* customer_name – This coulmn contains specific customer unique id under it
* customer_emailid – Customer’s Email Address
* order_date – Date on which the order was placed
* order_fruits –Fruits ordered per transaction
* order_vegetables-Vegetables ordered per transaction
* order_milk-Milk ordered per transaction
* order_rice – Rice ordered per transaction
* basket – Total items ordered per transaction





In [18]:
data.isna().sum()

order_id                1
customer_name           0
customer_emailid        0
order_date              0
order_fruits        17803
order_vegetables    20998
order_milk          35349
order_rice          35952
basket                610
dtype: int64

In [9]:
data.shape

(108033, 9)

In [4]:
# 108033 records out of which 610 have empty basket and needs to be removed
df = data[data['basket'].notna()]
#this removes 610 rows which had no transaction at all

In [20]:
df.shape

(107423, 9)

In [5]:
order = df.iloc[: , 4:8]  #took 4 columns on which we will be working
list_item=[]

In [71]:
order.head()

,order_fruits,order_vegetables,order_milk,order_rice
0,"Black sapota-1kg, Custard apple-1kg, Kamiya pa...","Gaajar-200g, Brocoli-250g",NaN,NaN
1,"Soursop Custard apple-1kg, Guinea Gold papaya-...","brown onion-5kg, Tomato Hybrid-2kg, red chilli...",NaN,NaN
2,"Green sapota-1kg, Soursop Custard apple-1kg, K...","brown onion-5kg, cherrie tomato-1kg, red bell ...",Buffalo Milk-1.5L,"Basmati Rice-3kg, Brown Rice-5kg"
3,"Dasheri mango-0.5kg, Dholka pomograntes-1kg, ...","Tomato Local-3kg, cherrie tomato-1kg, red chil...",Buffalo Milk-1.5L,Brown Rice-5kg
4,NaN,"Spring onion-250g, Tomato Hybrid-2kg, cherrie ...",Buffalo Milk-1.5L,Basmati Rice-3kg


In [6]:
#we took all the items per transaction and made a list for it
for i in range(len(order)):
    list_item.append(list(order.iloc[i,:]))

In [9]:
#sample 
list_item[:1][0][0]

'Black sapota-1kg, Custard apple-1kg, Kamiya papaya-1pc'

In [10]:
for i in range(len(list_item)):
    list_item[i]=[str(x) for x in list_item[i]]
    list_item[i]=[x for x in list_item[i] if x != 'nan'] #Removing nan values inside the list

In [11]:
# here the list is cleaned with na and quantity and we will also be able see how many items purchased per transactions. 
list_itemnames=[] 
       
for i in range(len(list_item)):
    s=[]
    for j in range(len(list_item[i])):
        a=list(list_item[i][j].split(","))
        for c in range(len(a)):
            v=list(a[c].partition("-"))
            s.append(v[0].lower())
    list_itemnames.append(s)

In [16]:
list_itemnames[:1][0]

['black sapota', ' custard apple', ' kamiya papaya', 'gaajar', ' brocoli']

In [17]:
xtemp=[[x.lower() for x in l] for l in list_itemnames]
xtemp2=[[x.replace('water melon' , 'watermelon') for x in l] for l in xtemp]
xtemp3=[[x.replace('indiagate basmati rice' , 'india gate basmati rice') for x in l] for l in xtemp2]

##### list cleaned and spell checked tried using Text Blob and Spell checker but they were not accurate

In [18]:
listt=[]    # a list which iterates inside the function and gets appended
temp_list2=[]         
for i in range(len(xtemp3)):
    listt=[]
    for j in range(len(xtemp3[i])):
        v=xtemp3[i][j].split(" ")
        if v[-1] =='pomograntes' or v[-1] =='pomogranates' or v[-1] == 'pomegranetes':
            s=[sub.replace(v[-1] ,'pomegranates') for sub in v]
            s=(" ").join(s)
        else:
            s=(" ").join(v)
            
        listt.append(s.lower().lstrip())
    temp_list2.append(listt)

In [153]:
temp_list2[:3]

[['black sapota', 'custard apple', 'kamiya papaya', 'gaajar', 'brocoli'],
 ['soursop custard apple',
  'guinea gold papaya',
  'kamiya papaya',
  'rasthali banana',
  'brown onion',
  'tomato hybrid',
  'red chilli',
  'gaajar',
  'yellow bell pepper'],
 ['green sapota',
  'soursop custard apple',
  'kamiya papaya',
  'brown onion',
  'cherrie tomato',
  'red bell pepper',
  'buffalo milk',
  'basmati rice',
  'brown rice']]

### Association Rules
* Association rules is a rule-based learning method used to draw frequent patterns and correlations from datasets such as transactional and relational data.

* Mlxtend is a rich and useful library for machine learning. It provides methods in association rules with a major algorithm apriori.


In [154]:
#creating dummy variables for items to build association rules

x= pd.Series(temp_list2)
d=pd.DataFrame(x)
d.columns = ['transaction']
df_association = d['transaction'].str.join(sep='*').str.get_dummies(sep='*')

In [156]:
df_association

,24 mantra brown rice,24 mantra organic brown rice,24 mantra organic sona masuri brown rice,24 mantra organic sona masuri polished rice,24 mantra organic sona masuri raw rice,24 mantra organic sona masuri rice,24 mantra sona masuri rice,agro fresh economy sona masuri rice,badmi mango,basmati brown rice,...,soursop custard apple,special basmati rice,spring onion,sweet soursop custard apple,tomato hybrid,tomato local,toned milk,vmd sona masuri rice supreme,watermelon,yellow bell pepper
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107418,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
107419,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
107420,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
107421,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [25]:
fp_grow=fpgrowth(df_association,min_support=0.2,use_colnames=True )
fp_grow.sort_values('support' , ascending = False , inplace = True )

In [26]:
fp_grow[:5]

,support,itemsets
19,0.500991,(watermelon)
15,0.378671,(green chilli)
12,0.373263,(carrot)
5,0.367957,(buffalo milk)
17,0.337349,(sona masuri rice)


In [27]:
fp_max=fpmax(df_association,min_support=0.2 , use_colnames=True)
fp_max.sort_values('support' , ascending=False , inplace=True)

In [28]:
fp_max[:5]

,support,itemsets
16,0.367957,(buffalo milk)
15,0.337349,(sona masuri rice)
14,0.328207,(capsicum)
13,0.308453,(tomato local)
12,0.289333,(basmati rice)


In [190]:
# Using Apriori algorithm and defining association rules
freqitemset = apriori(df_association , min_support = 0.05 , max_len = 2  , use_colnames = True )
freqitemset.sort_values('support' , ascending = False , inplace = True )

rules = association_rules(freqitemset , metric = 'lift' , min_threshold=1.5) 

In [212]:
rules.sort_values('lift',ascending=False,inplace=True)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
56,(dasheri mango),(badmi mango),0.203178,0.172952,0.080467,0.396041,2.289895,0.045327,1.369379
57,(badmi mango),(dasheri mango),0.172952,0.203178,0.080467,0.465256,2.289895,0.045327,1.490101
72,(dasheri mango),(dholka pomegranates),0.203178,0.153496,0.066624,0.327912,2.136288,0.035437,1.259513
73,(dholka pomegranates),(dasheri mango),0.153496,0.203178,0.066624,0.434047,2.136288,0.035437,1.407929
95,(kamiya papaya),(sweet soursop custard apple),0.167683,0.152072,0.051860,0.309277,2.033755,0.026361,1.227595
...,...,...,...,...,...,...,...,...,...
81,(rasthali banana),(red onion),0.144541,0.281969,0.061458,0.425195,1.507947,0.020702,1.249172
85,(basmati rice),(black sapota),0.289333,0.138527,0.060332,0.208520,1.505262,0.020251,1.088432
84,(black sapota),(basmati rice),0.138527,0.289333,0.060332,0.435522,1.505262,0.020251,1.258981
45,(brocoli),(custard apple),0.263491,0.230118,0.091191,0.346087,1.503954,0.030557,1.177346


In [221]:
rules[ (rules['lift'] >= 2) & (rules['confidence'] >= 0.35) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
56,(dasheri mango),(badmi mango),0.203178,0.172952,0.080467,0.396041,2.289895,0.045327,1.369379
57,(badmi mango),(dasheri mango),0.172952,0.203178,0.080467,0.465256,2.289895,0.045327,1.490101
73,(dholka pomegranates),(dasheri mango),0.153496,0.203178,0.066624,0.434047,2.136288,0.035437,1.407929


In [19]:
customer=df.iloc[ : , 1]
#list of all the customer id present in the data

In [20]:
customer=customer.reset_index()
customer=customer.iloc[:,1]
customer
# index sorted 

0          CUST189
1          CUST230
2          CUST289
3          CUST193
4          CUST234
            ...   
107418    CUST286Q
107419     CUST87Q
107420    CUST293Q
107421    CUST222Q
107422    CUST187Q
Name: customer_name, Length: 107423, dtype: object

In [21]:
all_items = [x for i in temp_list2 for x in i]     #list of all the items one by one

In [22]:
customer_list=[]

for i in range(len(temp_list2)):
    for j in range(len(temp_list2[i])):
         customer_list.append(customer[i])

#here we have now all the items and all the customer ids mapped together

### User to User Filtering

* User-Based Collaborative Filtering is a technique used to predict the items that a user might like on the basis of ratings or product purchased by the other users who have similar taste with that of the target user.

* This can achieved using cosine similarity between the users

In [162]:
final_data = pd.DataFrame(all_items)
final_data['customer']=customer_list
final_data.columns=['items' , 'customer']
# dataframe created with all the items with their customer ids

In [163]:
final_data.head()

,items,customer
0,black sapota,CUST189
1,custard apple,CUST189
2,kamiya papaya,CUST189
3,gaajar,CUST189
4,brocoli,CUST189


In [164]:
#6966 by 117 . Contains unique items and customers
all_dummy=pd.get_dummies(final_data , columns=['items'])
unique_dummy= pd.get_dummies(final_data, columns=['items']).groupby('customer', as_index=False).max()

In [165]:
unique_dummy.columns=unique_dummy.columns.str.replace('items','')
unique_dummy.columns=unique_dummy.columns.str.replace('_','')

In [166]:
unique_dummy

,customer,24 mantra brown rice,24 mantra organic brown rice,24 mantra organic sona masuri brown rice,24 mantra organic sona masuri polished rice,24 mantra organic sona masuri raw rice,24 mantra organic sona masuri rice,24 mantra sona masuri rice,agro fresh economy sona masuri rice,badmi mango,...,soursop custard apple,special basmati rice,spring onion,sweet soursop custard apple,tomato hybrid,tomato local,toned milk,vmd sona masuri rice supreme,watermelon,yellow bell pepper
0,CUST1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,CUST10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,CUST100,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
3,CUST100A,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
4,CUST100B,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6961,CUST9M,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
6962,CUST9N,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
6963,CUST9O,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
6964,CUST9P,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [168]:
unique_dummy.shape   #shape of the matrix 6966 by 117

(6966, 117)

In [167]:
cosine_sim = 1-pairwise_distances(unique_dummy.iloc[:,1:], metric="cosine")
#similarity measure for users

In [169]:
cosine_sim[:10]

array([[1.        , 0.38461538, 0.28644595, ..., 0.20965697, 0.20965697,
        0.20965697],
       [0.38461538, 1.        , 0.14322297, ..., 0.20965697, 0.20965697,
        0.20965697],
       [0.28644595, 0.14322297, 1.        , ..., 0.29277002, 0.29277002,
        0.29277002],
       ...,
       [0.2773501 , 0.13867505, 0.90369611, ..., 0.28347335, 0.28347335,
        0.28347335],
       [0.28644595, 0.14322297, 0.93333333, ..., 0.29277002, 0.29277002,
        0.29277002],
       [0.2773501 , 0.13867505, 0.90369611, ..., 0.28347335, 0.28347335,
        0.28347335]])

In [170]:
cosine_sim.shape

(6966, 6966)

In [171]:
custindex=pd.Series(unique_dummy.index , index=unique_dummy['customer']) #creating index for customers
cust_id=custindex['CUST1']
cust_id

0

In [172]:
def recommend(cust , topn):
    cust_id=custindex[cust]
    cosscore=list(enumerate(cosine_sim[cust_id]))
    cosscore=sorted(cosscore , key=lambda x:x[1] , reverse=True)
    cosscore_new=cosscore[0:topn+1]
    
    cust_idx=[i[0] for i in cosscore_new]
    cust_score=[i[1] for i in cosscore_new]
    
    similar_cust=pd.DataFrame(columns=['Name' , 'Scores'])
    similar_cust['Name']=unique_dummy.loc[cust_idx , 'customer']
    similar_cust['Scores']=cust_score
    print(similar_cust)
    

recommend('CUST1' , topn=10)   #need top 10 similar customers for CUST1

        Name    Scores
0      CUST1  1.000000
1988  CUST1H  0.960769
1995  CUST1O  0.960769
1996  CUST1P  0.960769
1985  CUST1E  0.919866
1986  CUST1F  0.919866
1987  CUST1G  0.919866
1989  CUST1I  0.919866
1990  CUST1J  0.919866
1991  CUST1K  0.919866
1993  CUST1M  0.919866


### Item to Item Filtering
#### Similar to user to user but here we use an item matrix to explore the relationship between pair of items using cosine similarity

In [174]:
item_dummy=unique_dummy.T
item_dummy=item_dummy.reset_index()
item_dummy.columns=item_dummy.iloc[0,:]
item_dummy=item_dummy.iloc[1:,:].reset_index()
item_dummy=item_dummy.iloc[:,1:]
item_dummy=item_dummy.rename(columns = {'customer':'items'})
item_dummy #items as rows and customers as columns

,items,CUST1,CUST10,CUST100,CUST100A,CUST100B,CUST100C,CUST100D,CUST100E,CUST100F,...,CUST9H,CUST9I,CUST9J,CUST9K,CUST9L,CUST9M,CUST9N,CUST9O,CUST9P,CUST9Q
0,24 mantra brown rice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24 mantra organic brown rice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,24 mantra organic sona masuri brown rice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,24 mantra organic sona masuri polished rice,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,24 mantra organic sona masuri raw rice,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,tomato local,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112,toned milk,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
113,vmd sona masuri rice supreme,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
114,watermelon,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
item_cosine =1-pairwise_distances(item_dummy.iloc[:,1:] , metric='cosine')
# similarity measure for items 

itemindex=pd.Series(item_dummy.index , index=item_dummy['items'])
itemindex.head()   #index for items

items
24 mantra brown rice                           0
24 mantra organic brown rice                   1
24 mantra organic sona masuri brown rice       2
24 mantra organic sona masuri polished rice    3
24 mantra organic sona masuri raw rice         4
dtype: int64

In [48]:
def item_recommend(item , topn):
    item_id=itemindex[item]
    cosscore=list(enumerate(item_cosine[item_id]))
    cosscore=sorted(cosscore , key=lambda x:x[1] , reverse=True)
    cosscore_new=cosscore[0:topn+1]
    
    item_idx=[i[0] for i in cosscore_new]
    item_score=[i[1] for i in cosscore_new]
    
    similar_item=pd.DataFrame(columns=['Name' , 'Scores'])
    similar_item['Name']=item_dummy.loc[item_idx , 'items']
    similar_item['Scores']=item_score
    print('Items similar to {} are '.format(item))
    print(similar_item)
    
    
item_recommend('organic sona masuri rice' , 5)
item_recommend(98, 5)

Items similar to organic sona masuri rice are 
                        Name    Scores
83  organic sona masuri rice  1.000000
75         more basmati rice  0.685994
57   india gate basmati rice  0.563602
52                gala apple  0.110337
74             mammey sapota  0.108078
94           rasthali banana  0.104063
Items similar to 98 are 
                  Name    Scores
98   ruby pomegranates  1.000000
113         watermelon  0.484706
53        green chilli  0.473712
96          red chilli  0.460119
22              carrot  0.457957
104   sona masuri rice  0.457746


In [175]:
unique_dum=unique_dummy.set_index('customer')

In [176]:
item_matrix=csr_matrix(unique_dum.values)

In [178]:
item_matrix

<6966x116 sparse matrix of type '<class 'numpy.uint8'>'
	with 108961 stored elements in Compressed Sparse Row format>

### Alternating Least Squares for Recommendations

* Implicit data (the type of data we’re using here) is data we gather from the users behaviour, with no ratings or specific actions needed. It could be what items a user purchased, how many times they played a song or watched a movie, how long they’ve spent reading a specific article etc.

In [51]:
matrix_size=item_matrix.shape[0]*item_matrix.shape[1]
num_purchases=len(item_matrix.nonzero()[0])
sparsity=100*(1-(num_purchases/matrix_size))
sparsity

86.56018674555918

In [179]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((item_matrix*alpha).astype('double'), 
                                                          factors=20, 
                                                          regularization = 0.1, 
                                                         iterations = 50)

This method is deprecated. Please use the AlternatingLeastSquares class instead


In [180]:
import numpy as np
customer_arr = np.array(range(0,6967))  # Array of customer IDs from the ratings matrix
products_arr=np.array(range(0,116)) # Array of product IDs from the ratings matrix

In [181]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list):
   
    cust_ind = np.where(customers_list == customer_id)[0][0] # Returns the index row of our customer id
    
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    print('Items which the {} has already bought are given below:'.format(custindex.index[customer_id]))
    return list(enumerate(itemindex.index[prod_codes]))

In [182]:
get_items_purchased(0, item_matrix, customer_arr, products_arr)

Items which the CUST1 has already bought are given below:


[(0, '24 mantra organic sona masuri raw rice'),
 (1, 'brocoli'),
 (2, 'carrot'),
 (3, 'custard apple'),
 (4, 'daawat'),
 (5, 'guinea gold papaya'),
 (6, 'kesar mango'),
 (7, 'premium basmati rice'),
 (8, 'ruby pomegranates'),
 (9, 'safe harvest sona masuri rice'),
 (10, 'soursop custard apple'),
 (11, 'tomato local'),
 (12, 'yellow bell pepper')]

In [186]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, num_items = 10):
  cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of customer id
  pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
  pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
  pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
  rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
  min_max = MinMaxScaler()
  rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
  recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
  product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
  rec_list = [] # start empty list to store items
  for index in product_idx:
    code = item_list[index]
    rec_list.append(code) # Append our descriptions to the list
    
  print('Items which are not bought by {} and most likely to purchase are below :'.format(custindex.index[customer_id]))
  return list(itemindex.index[rec_list])

In [187]:
rec_items(0, item_matrix, user_vecs, item_vecs, customer_arr, products_arr,
                       num_items = 10)

Items which are not bought by CUST1 and most likely to purchase are below :


['sona masuri boiled rice',
 'daawat sona masuri rice',
 'toned milk',
 'buffalo milk',
 'sweet soursop custard apple',
 'kohinoor authentic brown basmati rice',
 'popular essentials basmati rice brown',
 'red bell pepper',
 'safe harvest unpolished brown sona masuri rice',
 'capsicum']